In [13]:
import cv2
import os
import time

image_dir = r'C:\Users\ishigaki\Downloads\archive\Original Images\Original Images' # ローカルの画像データファイルパス


In [14]:
processed_images = []
labels = []
label_name = []
haarcascade_path = r'data\haarcascade_frontalface_default.xml'
print("Using Haar Cascade path:", haarcascade_path)
if not os.path.exists(haarcascade_path):
    raise FileNotFoundError(f'Haar Cascade XML file not found at {haarcascade_path}')
face_cascade = cv2.CascadeClassifier(haarcascade_path)

Using Haar Cascade path: data\haarcascade_frontalface_default.xml


In [15]:
for label, person_name in enumerate(os.listdir(image_dir)):
    person_dir = os.path.join(image_dir, person_name)
    if not os.path.isdir(person_dir):
        continue

    for image_name in os.listdir(person_dir):
        image_path = os.path.join(person_dir, image_name)
        image = cv2.imread(image_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            face = gray_image[y:y+h, x:x+w]
            resized_face = cv2.resize(face, (128, 128))
            processed_images.append(resized_face)
            labels.append(label)
    label_name.append(person_name)

In [16]:
print(label_name)

['Akshay Kumar', 'Alexandra Daddario', 'Alia Bhatt', 'Amitabh Bachchan', 'Andy Samberg', 'Anushka Sharma', 'Billie Eilish', 'Brad Pitt', 'Camila Cabello', 'Charlize Theron', 'Claire Holt', 'Courtney Cox', 'Dwayne Johnson', 'Elizabeth Olsen', 'Ellen Degeneres', 'Henry Cavill', 'Hrithik Roshan', 'Hugh Jackman', 'Jessica Alba', 'Kashyap', 'Lisa Kudrow', 'Margot Robbie', 'Marmik', 'Natalie Portman', 'Priyanka Chopra', 'Robert Downey Jr', 'Roger Federer', 'Tom Cruise', 'Vijay Deverakonda', 'Virat Kohli', 'Yudai', 'Zac Efron']


In [17]:
import tensorflow as tf
from keras import layers
from keras import models
import numpy as np
import tensorflow as tf



model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(set(labels)), activation='softmax')  # Number of unique labels
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

processed_images = np.array(processed_images).reshape(-1, 128, 128, 1)
labels = np.array(labels)

model.fit(processed_images, labels, epochs=10, validation_split=0.2)

model.save(r'C:\Users\ishigaki\Documents\imageAI.h5')

Epoch 1/10


c:\Users\ishigaki\Documents\imageAI\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


105/105 ━━━━━━━━━━━━━━━━━━━━ 19s 173ms/step - accuracy: 0.0786 - loss: 16.1117 - val_accuracy: 0.0000e+00 - val_loss: 7.9871
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 17s 164ms/step - accuracy: 0.2860 - loss: 2.5922 - val_accuracy: 0.0060 - val_loss: 8.2797
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 196ms/step - accuracy: 0.4461 - loss: 2.0444 - val_accuracy: 0.0048 - val_loss: 13.7564
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 196ms/step - accuracy: 0.6383 - loss: 1.3566 - val_accuracy: 0.0095 - val_loss: 16.9385
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 197ms/step - accuracy: 0.7208 - loss: 1.0183 - val_accuracy: 0.0095 - val_loss: 24.0128
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 197ms/step - accuracy: 0.7809 - loss: 0.8170 - val_accuracy: 0.0084 - val_loss: 29.8271
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 197ms/step - accuracy: 0.8078 - loss: 0.7177 - val_accuracy: 0.0084 - val_loss: 38.5196
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 196ms/step - accuracy: 0.8282 - loss: 0.

In [18]:
def recognize_face(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = gray_image[y:y+h, x:x+w]
        resized_face = cv2.resize(face, (128, 128)).reshape(1, 128, 128, 1)
        predictions = model.predict(resized_face)
        label = np.argmax(predictions)
        confidence = np.max(predictions)

        return label, confidence

    return None, None

In [25]:
video_capture = cv2.VideoCapture(0)
Absence = None
name = None
while True:
    ret, frame = video_capture.read()
    label, confidence = recognize_face(frame)

    # if label == 0:
    #     name = "Yudai" a
    if confidence == None:
        if Absence == None:
            Absence = time.time()
        elif time.time() - Absence >= 5:
            break
    elif confidence > 0.7:  # しきい値
        Absence = None
        print(f"検出した人物名: {label_name[label]}")
        print(f"一致率: {confidence}")

video_capture.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
検出した人物名: Claire Holt
一致率: 0.7956508994102478
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
検出した人物名: Claire Holt
一致率: 0.7967464327812195
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
検出した人物名: Andy Samberg
一致率: 0.9999926090240479
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
検出した人物名: Andy Samberg
一致率: 0.9953925609588623
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
検出した人物名: Andy Samberg
一致率: 0.9839962124824524
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
検出した人物名: Andy Samberg
一致率: 0.8119254112243652
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
検出した人物名: Andy Samberg
一致率: 0.9905189871788025
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
検出した人物名: Andy Samberg
一致率: 0.999863862991333
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
検出した人物名: Andy Samberg
一致率: 0.9997060894966125
1/1 ━━━━━━━━━━━━━━━

In [24]:
test_dir = r'C:\Users\ishigaki\Downloads\test'
for image_name in os.listdir(test_dir):
    label, confidence = recognize_face(frame)
    image_path = os.path.join(test_dir, image_name)
    image = cv2.imread(image_path)
    
    label, confidence = recognize_face(image)
    print(f"検出した人物名: {label_name[label]}")
    print(f"一致率: {confidence}")
    print(f"ファイル名: {image_name}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
検出した人物名: Camila Cabello
一致率: 0.9999929666519165
ファイル名: 45x35_web.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
検出した人物名: Alia Bhatt
一致率: 0.999984860420227
ファイル名: alia-bhatt1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
検出した人物名: Alia Bhatt
一致率: 0.9374845623970032
ファイル名: alia-bhatt2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
検出した人物名: Natalie Portman
一致率: 0.5234331488609314
ファイル名: elizabeth olsen.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
検出した人物名: Hugh Jackman
一致率: 0.942503809928894
ファイル名: hugh-jackman2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
検出した人物名: Camila Cabello
一致率: 0.35500362515449524
ファイル名: idphoto.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
検出した人物名: Lisa Kudrow
一致率: 0.9999793767929077
ファイル名: 